In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import json
import os
from os import path
from os.path import join
import subprocess
import time
import datetime
import numpy as np

def run(cmd_list, tail_lines=0):
    return '\n'.join(subprocess.check_output(cmd_list).decode('utf-8').split('\n')[- tail_lines:]).strip()

def head(file, lines):
    return subprocess.check_output(['head', '-n', str(lines), file]).decode('utf-8')

def tail(file, lines):
    return subprocess.check_output(['tail', '-n', str(lines), file]).decode('utf-8')

def get_logfiles_by_pattern(path, pattern):
#     files = subprocess.check_output(['find', path, '-cmin', '-%s' % age_minutes]).decode('utf-8').split('\n')
    cmd_list = ['ls', path]
    print(cmd_list)
    files = subprocess.check_output(cmd_list).decode('utf-8').split('\n')
    files = [f for f in files if f != '' and not f.endswith('logs') and pattern in f]
    return files

def get_num_lines(filepath):
    num_lines = int(subprocess.check_output(['wc', '-l', filepath]).decode('utf-8').split(' ')[0])
    return num_lines

def get_meta_keys(filepath, keys):    
    try:
        with open(filepath, 'r') as f:
            meta_line = f.readline()
        meta = json.loads(meta_line.replace('meta: ', '').strip())
#         print(meta)
#         params = meta.get('params', {})
        values = []
        for key in keys:
            d = meta
#             print('d', d)
            key_parts = key.split('.')
            for k in key_parts[:-1]:
#                 print('k', k)
#                 print('d.keys()', d.keys(), 'k', k)
                d = d.get(k, {})
#                 print('d', d)
            v = d.get(key_parts[-1], '')
            values.append(v)
        return values
    except Exception as e:
        print('exception', e, filepath)
        return [None] * len(keys)

def print_git_info(filepath):
    gitlog, gitdiff = get_meta_keys('../logs/' + log, ['gitlog', 'gitdiff'])
    print(gitlog)
    print(gitdiff)
    
def get_cmd_line(filepath):
    cmd_line = get_meta_keys('../logs/' + log, ['argv'])[0]
    return ' '.join(cmd_line)
    
print(get_cmd_line('../logs/log_20180805_164700_latlang_hughtok2.log'))
print_git_info('../logs/log_20180805_164700_latlang_hughtok2.log')

# logs = get_logfiles_by_pattern('../logs', 'latlang')
# # print(logs)
# for log in logs:
#     lines = get_num_lines('../logs/' + log)
#     if lines <= 10:
#         continue
#     argv = get_meta_keys('../logs/' + log, ['argv'])
#     last_line = tail('../logs/' + log, lines=1)
#     print(last_line)
#     die()
#     last_line_d = json.loads(last_line)
#     print(log, lines, 'e=', last_line_d['episode'], ' '.join(argv[0]))
#     print(last_line_d.keys())
#     print('params', params)
#     print(params['argv'])